<a href="https://colab.research.google.com/github/Ramyrahmeni/DeepLearning/blob/main/notebook6a21d4412b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

ramyrahmeni_basedata_path = kagglehub.dataset_download('ramyrahmeni/basedata')
ramyrahmeni_u1test_path = kagglehub.dataset_download('ramyrahmeni/u1test')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/basedata/u1.base


In [ ]:
import pandas as pd

# Charger le fichier de données
column_names = ['user_id', 'movie_id', 'rating', 'timestamp']
train_data = pd.read_csv('/kaggle/input/basedata/u1.base', sep='\t', names=column_names)

# Afficher un aperçu des données
print(train_data.head())


# Display the first few rows of the training data
train_data.head()


   user_id  movie_id  rating  timestamp
0        1         1       5  874965758
1        1         2       3  876893171
2        1         3       4  878542960
3        1         4       3  876893119
4        1         5       3  889751712


,user_id,movie_id,rating,timestamp
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712


In [ ]:
vote_matrix = train_data.pivot(index='user_id', columns='movie_id', values='rating')
vote_matrix

movie_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,NaN,4.0,1.0,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,2.0,NaN,NaN,4.0,5.0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,5.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import cupy as cp
from tqdm import tqdm

def pearson_similarity(user1, user2, min_common=7):
    # Convert numpy arrays to cupy arrays
    user1 = cp.asarray(user1)
    user2 = cp.asarray(user2)

    # Identify common (non-NaN) ratings
    common_ratings = ~cp.isnan(user1) & ~cp.isnan(user2)
    n_common = common_ratings.sum()

    if n_common < min_common:
        return cp.nan

    # Compute Pearson correlation using common ratings only
    return cp.corrcoef(user1[common_ratings], user2[common_ratings])[0, 1]

n_users = vote_matrix.shape[0]
similarity_matrix = cp.zeros((n_users, n_users))

# Wrap the outer loop with tqdm to show progress
for i in tqdm(range(n_users), desc="Calculating similarity matrix"):
    for j in range(i + 1, n_users):
        similarity = pearson_similarity(vote_matrix.iloc[i].values, vote_matrix.iloc[j].values)
        similarity_matrix[i, j] = similarity
        similarity_matrix[j, i] = similarity

similarity_matrix = cp.nan_to_num(similarity_matrix, nan=0)
print(cp.asnumpy(similarity_matrix))  # Convert back to numpy if needed for display


Calculating similarity matrix: 100%|██████████| 943/943 [07:23<00:00,  2.13it/s]


[[ 0.          0.          0.         ...  0.67936622 -0.19348921
   0.24303845]
 [ 0.          0.          0.         ...  0.          0.25943726
   0.62749502]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 ...
 [ 0.67936622  0.          0.         ...  0.          0.
   0.        ]
 [-0.19348921  0.25943726  0.         ...  0.          0.
   0.27743263]
 [ 0.24303845  0.62749502  0.         ...  0.          0.27743263
   0.        ]]


In [ ]:
import cupy as cp
from tqdm import tqdm

def get_k_nearest_neighbors(user_index, similarity_matrix, k=5):
    # Get similarities for the specified user
    user_similarities = similarity_matrix[user_index]

    # Eliminate negative similarities by setting them to 0
    user_similarities[user_similarities < 0] = 0

    # Get indices of sorted similarities in descending order
    sorted_indices = cp.argsort(user_similarities)[::-1]

    # Exclude the user itself and select top k neighbors
    nearest_neighbors = [idx for idx in sorted_indices if idx != user_index][:k]

    return nearest_neighbors

# Example usage with progress bar
k = 5  # Set the number of neighbors
n_users = similarity_matrix.shape[0]
all_neighbors = {}

# Wrap the loop with tqdm to show progress
for user_index in tqdm(range(n_users), desc="Finding k nearest neighbors"):
    all_neighbors[user_index] = get_k_nearest_neighbors(user_index, similarity_matrix, k)

print(all_neighbors)  # Contains k-nearest neighbors for each user


Finding k nearest neighbors: 100%|██████████| 943/943 [01:05<00:00, 14.36it/s]


{0: [array(226), array(68), array(328), array(164), array(36)], 1: [array(233), array(192), array(932), array(914), array(650)], 2: [array(444), array(545), array(594), array(555), array(675)], 3: [array(723), array(682), array(626), array(918), array(591)], 4: [array(583), array(25), array(56), array(20), array(500)], 5: [array(873), array(266), array(247), array(21), array(30)], 6: [array(908), array(851), array(936), array(469), array(581)], 7: [array(830), array(937), array(196), array(902), array(396)], 8: [array(631), array(795), array(879), array(456), array(654)], 9: [array(120), array(838), array(151), array(194), array(41)], 10: [array(528), array(25), array(838), array(222), array(448)], 11: [array(290), array(229), array(329), array(94), array(747)], 12: [array(170), array(608), array(172), array(655), array(809)], 13: [array(479), array(886), array(255), array(660), array(745)], 14: [array(261), array(262), array(108), array(358), array(611)], 15: [array(229), array(263), 

In [ ]:
def predict_rating(user_index, item_index, vote_matrix, similarity_matrix, neighbors):
    # Filter neighbors who have rated the item
    neighbors_who_rated = [n for n in neighbors if not np.isnan(vote_matrix.iloc[n, item_index])]

    if len(neighbors_who_rated) == 0:
        return np.nan  # Return NaN if no neighbors have rated the item

    # Calculate weighted average
    num = sum(similarity_matrix[user_index, n] * vote_matrix.iloc[n, item_index] for n in neighbors_who_rated)
    den = sum(similarity_matrix[user_index, n] for n in neighbors_who_rated)

    return num / den if den != 0 else np.nan


In [ ]:
# Define the user index for which to generate predictions
user_index = 3  # Replace with the desired user index

# Retrieve the k nearest neighbors for the specified user and cast to integers
neighbors = [int(n) for n in get_k_nearest_neighbors(user_index, similarity_matrix, k=5)]

# Predict ratings for the specified user
user_predictions = predict_rating(user_index, ,vote_matrix, similarity_matrix, neighbors)

print(user_predictions)  # Contains predicted ratings for this user's unknown items


In [ ]:
from tqdm import tqdm

# Define the function to fill NaN values in the vote matrix with progress tracking
def fill_missing_ratings(vote_matrix, similarity_matrix, k=5):
    filled_matrix = vote_matrix.copy()  # Create a copy to avoid modifying the original matrix

    # Progress bar for users
    for user_index in tqdm(range(vote_matrix.shape[0]), desc="Filling missing ratings by user"):
        # Progress bar for items within each user
        for item_index in range(vote_matrix.shape[1]):
            # Check if the current value is NaN
            if np.isnan(vote_matrix.iloc[user_index, item_index]):
                # Get the k nearest neighbors for the specified user
                neighbors = [int(n) for n in all_neighbors[user_index]]

                # Predict the rating for the missing value
                predicted_rating = predict_rating(user_index, item_index, vote_matrix, similarity_matrix, neighbors)

                # Update the missing value with the predicted rating if a prediction is available
                if not np.isnan(predicted_rating):
                    filled_matrix.iat[user_index, item_index] = predicted_rating

    return filled_matrix

# Use the function to fill NaN values in the vote_matrix with progress tracking
filled_vote_matrix = fill_missing_ratings(vote_matrix, similarity_matrix, k=5)

print(filled_vote_matrix)  # The vote matrix with NaN values filled by predicted ratings


Filling missing ratings by user: 100%|██████████| 943/943 [09:58<00:00,  1.58it/s]

movie_id      1     2     3         4     5     6         7     8     \
user_id                                                                
1         5.000000   3.0   4.0  3.000000   3.0   NaN  4.000000   1.0   
2         4.000000   2.0   NaN  3.517010   3.0   NaN  4.000000   NaN   
3         3.936395   NaN   4.0       NaN   5.0   NaN  2.996848   NaN   
4         4.000000   3.0   4.0  2.299524   4.0   NaN  4.320614   5.0   
5         4.286085   NaN   NaN       NaN   2.0   NaN  4.502677   4.0   
...            ...   ...   ...       ...   ...   ...       ...   ...   
939       3.629760   NaN   NaN       NaN   NaN   NaN  4.000000   3.0   
940            NaN   NaN   NaN  2.000000   4.0   NaN  4.000000   5.0   
941       5.000000   NaN   1.0       NaN   NaN   NaN  4.000000   4.0   
942            NaN   NaN   NaN       NaN   4.0   NaN  4.000000   5.0   
943       4.000000   5.0   NaN       NaN   NaN   NaN  3.483513   NaN   

movie_id      9     10    ...  1673  1674  1675  1676  1677  16

In [ ]:
filled_vote_matrix

movie_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.000000,3.0,4.0,3.000000,3.0,NaN,4.000000,1.0,5.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.000000,2.0,NaN,3.517010,3.0,NaN,4.000000,NaN,3.000000,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.936395,NaN,4.0,NaN,5.0,NaN,2.996848,NaN,2.666115,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.000000,3.0,4.0,2.299524,4.0,NaN,4.320614,5.0,4.659302,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.286085,NaN,NaN,NaN,2.0,NaN,4.502677,4.0,5.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,3.629760,NaN,NaN,NaN,NaN,NaN,4.000000,3.0,5.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,2.000000,4.0,NaN,4.000000,5.0,3.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,5.000000,NaN,1.0,NaN,NaN,NaN,4.000000,4.0,2.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
vote_matrix

movie_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,NaN,4.0,1.0,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,2.0,NaN,NaN,4.0,5.0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,5.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
